In [ ]:
# 딥러닝 회귀
# 출력의 개수를 1개로
# 손실 함수를 MSE나 기타 등등...
# 데이터셋과 데이터 로드를 커스텀하게 정의해서 사용
# 나머지는 동일한 패턴으로 학습 평가

In [10]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

data_url = 'http://lib.stat.cmu.edu/datasets/boston'
raw_df = pd.read_csv(data_url,sep="\s+",skiprows=22,header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]


class BostonDataSet(Dataset) :
    def __init__(self,X,y) :
        self.X = torch.tensor(X,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32).view(-1,1)
    def __len__(self) :
        return len(self.X)
    def __getitem__(self,idx) :
        return self.X[idx],self.y[idx]

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\31799\AppData\Local\Temp\ipykernel_15824\2079271604.py:8: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url,sep="\s+",skiprows=22,header=None)


In [11]:
X_dataset = BostonDataSet(data,target)
X_train_loader = DataLoader(X_dataset,batch_size=32,shuffle=True)

In [12]:
# 회긔 모델 정의
class BostonRegresstion(nn.Module):
    def __init__(self, input_dim):
        super(BostonRegresstion,self).__init__()
        self.model =  nn.Sequential(
            nn.Linear(input_dim,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,1)
        )
    def forward(self,x):
        return self.model(x)

In [14]:
from torch.optim import Adam
model = BostonRegresstion(data.shape[1])
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)

In [18]:
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
epochs = 20
# 학습루프
for epoch in range(epochs) :
    tqdm_obj = tqdm(total=len(X_train_loader),desc=f"Epoch {epoch+1}/{epochs}")
    loss_list = 0
    for data, label in tqdm(X_train_loader) :
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = criterion(y_pred,label)
        loss_list += loss.item()
        loss.backward()
        optimizer.step()

        tqdm_obj.set_postfix({'loss':loss.item()})

    tqdm_obj.close()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss_list/len(X_train_loader):.4f}")

torch.save(model.state_dict(),'./data/boston_model.pth')

Epoch 1/20:   0%|          | 0/16 [00:00<?, ?it/s, loss=40.6]


Epoch [1/20], Loss: 24.5370


Epoch 2/20:   0%|          | 0/16 [00:00<?, ?it/s, loss=16]


Epoch [2/20], Loss: 25.8558


Epoch 3/20:   0%|          | 0/16 [00:00<?, ?it/s, loss=20.6]


...

Epoch [18/20], Loss: 21.6836


Epoch 19/20:   0%|          | 0/16 [00:00<?, ?it/s, loss=23.1]


Epoch [19/20], Loss: 21.9075


Epoch 20/20:   0%|          | 0/16 [00:00<?, ?it/s, loss=26.9]


Epoch [20/20], Loss: 21.4602


In [ ]:
# 평가
model.eval()
total_loss = 0.0
total_correct = 0
total_samples = 0

criterion = nn.MSELoss()
with torch.no_grad():
    for data, label in tqdm(X_train_loader,desc ="Evaluating"):
        data, label = data.to